# 🎛️ Professional Mixing Session
## Multi-Channel Mixing → Mixed WAV Export

This notebook provides professional mixing of raw audio channels, creating a balanced stereo mix.

**Workflow:**
1. Load raw channels (organized by instrument groups)
2. Apply professional mixing (EQ, compression, spatial positioning, effects)
3. Export final mixed WAV file
4. Optionally export stems for further processing

In [1]:
# Core imports and setup
import os
import json
import numpy as np
import soundfile as sf
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our mixing system
from mixing_engine import MixingSession, ChannelStrip, MixBus
from channel_recognition import identify_channel_type, suggest_processing
from mix_templates import MixTemplate, get_template

print("🎛️ Professional Mixing System Ready!")
print("📁 Supports multi-channel input with intelligent processing")
print("🎚️ Exports mixed WAV file")

DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
🎛️ Mixing Engine loaded!
   • Multi-channel support with intelligent routing
   • Template-based processing
   • Bus architecture with grouping
   • Exports stems for post-mix pipeline
🔍 Channel Recognition System loaded!
   • Name pattern matching with 40+ instrument types
   • Frequency analysis for content identification
   • Automatic processing suggestions
   • User hint support for ambiguous channels
🎨 Mix Templates loaded!
   • 5 genre-specific templates (Pop, Rock, EDM, Hip-Hop, Jazz)
   • Channel-specific EQ, compression, and effects
   • Intelligent spatial po

## 📁 Step 1: Define Your Input Channels

Organize your raw audio files by instrument category.

In [2]:
# Load from folder structure
def load_from_folder_structure(base_path: str) -> Dict:
    """Load channels from organized folder structure"""
    channels = {}
    base = Path(base_path)
    
    print(f"🔍 Looking for channels in: {base_path}")
    
    if not base.exists():
        print(f"❌ Directory not found: {base_path}")
        print("📝 Please update the base_path variable below with your actual channel directory")
        return {}
    
    # Expected categories
    categories = ['drums', 'bass', 'guitars', 'keys', 'vocals', 'backvocals', 'synths', 'strings', 'brass', 'percussion', 'fx', 'other']
    
    for category in categories:
        category_path = base / category
        if category_path.exists():
            channels[category] = {}
            audio_files = list(category_path.glob('*.wav'))
            if audio_files:
                print(f"  ✅ Found {category}/ with {len(audio_files)} files")
                for audio_file in audio_files:
                    channel_name = audio_file.stem
                    channels[category][channel_name] = str(audio_file)
            else:
                print(f"  ⚠️ Found {category}/ but no .wav files")
    
    return channels

# ⚠️ UPDATE THIS PATH TO YOUR ACTUAL CHANNEL DIRECTORY! ⚠️
base_path = "/Users/itay/Documents/post_mix_data/pre_mix_channels/combined_inst/"

print("📁 IMPORTANT: Update the base_path above with your actual directory!")
print("   Expected structure: your_directory/drums/kick.wav, your_directory/vocals/lead.wav, etc.")
print()

channels = load_from_folder_structure(base_path)

# Display loaded channels
def display_channels(channels):
    print("📁 Loaded Channels:")
    total = 0
    for category, tracks in channels.items():
        if tracks:
            print(f"\n  {category.upper()}:")
            for name, path in tracks.items():
                print(f"    • {name}: {os.path.basename(path)}")
                total += 1
    
    if total == 0:
        print("\n❌ NO CHANNELS LOADED!")
        print("\n🛠️ To fix this:")
        print("   1. Update the base_path variable above with your actual directory")
        print("   2. Make sure your audio files are organized like:")
        print("      your_directory/drums/kick.wav")
        print("      your_directory/bass/bass.wav")  
        print("      your_directory/vocals/lead.wav")
        print("   3. Re-run this cell")
        print("\n⚠️ Without channels, the mixer will create empty/silent files!")
    else:
        print(f"\n✅ Ready to mix {total} channels!")
    
    return total

total_channels = display_channels(channels)

# Safety check
if total_channels == 0:
    print("\n🛑 STOP: No channels loaded - please fix the path before continuing!")

📁 IMPORTANT: Update the base_path above with your actual directory!
   Expected structure: your_directory/drums/kick.wav, your_directory/vocals/lead.wav, etc.

🔍 Looking for channels in: /Users/itay/Documents/post_mix_data/pre_mix_channels/combined_inst/
  ✅ Found drums/ with 5 files
  ✅ Found bass/ with 5 files
  ✅ Found guitars/ with 6 files
  ✅ Found keys/ with 4 files
  ✅ Found vocals/ with 3 files
  ✅ Found backvocals/ with 5 files
  ✅ Found synths/ with 3 files
  ✅ Found fx/ with 6 files
📁 Loaded Channels:

  DRUMS:
    • tom: tom.wav
    • hihat: hihat.wav
    • kick: kick.wav
    • snare: snare.wav
    • cymbal: cymbal.wav

  BASS:
    • bass_guitar5: bass_guitar5.wav
    • bass1: bass1.wav
    • bass_guitar3: bass_guitar3.wav
    • bass_synth2: bass_synth2.wav
    • bass_synth4: bass_synth4.wav

  GUITARS:
    • electric_guitar4: electric_guitar4.wav
    • electric_guitar5: electric_guitar5.wav
    • electric_guitar6: electric_guitar6.wav
    • electric_guitar2: electric_guita

## 🎚️ Step 2: Choose Mix Template

In [3]:
# Select template
selected_template = "modern_pop"  # Change this to your genre

# Optional: Customize template parameters
template_customization = {
    "brightness": 0.7,      # 0-1 (dark to bright)
    "width": 0.8,          # 0-1 (mono to wide)
    "aggression": 0.8,     # 0-1 (gentle to aggressive)
    "vintage": 0.3,        # 0-1 (modern to vintage)
    "dynamics": 0.4,       # 0-1 (compressed to dynamic)
    "depth": 0.7,          # 0-1 (flat to deep)
}

print(f"🎛️ Selected Template: {selected_template}")
print("\n📊 Template Characteristics:")
for param, value in template_customization.items():
    bar = '█' * int(value * 10) + '░' * int((1-value) * 10)
    print(f"  {param:12} [{bar}] {value:.1%}")

🎛️ Selected Template: modern_pop

📊 Template Characteristics:
  brightness   [███████░░░] 70.0%
  width        [████████░] 80.0%
  aggression   [████████░] 80.0%
  vintage      [███░░░░░░░] 30.0%
  dynamics     [████░░░░░░] 40.0%
  depth        [███████░░░] 70.0%


## 🎛️ Step 3: Configure Mix Settings

In [4]:
# 🎚️ MIX BALANCE PRESETS 🎚️
# Choose a preset or create your own custom balance

# Preset 1: Vocal-Forward (for singer-songwriter, pop vocals)
vocal_forward = {
    "vocal_prominence": 0.8,     # Prominent vocals
    "drum_punch": 0.4,           # Subtle drums
    "bass_foundation": 0.5,      # Balanced bass
    "instrument_presence": 0.3,  # Background instruments
}

# Preset 2: Drum-Heavy (for rock, metal, energetic tracks)
drum_heavy = {
    "vocal_prominence": 0.5,     # Balanced vocals
    "drum_punch": 0.9,           # Punchy, aggressive drums
    "bass_foundation": 0.7,      # Strong foundation
    "instrument_presence": 0.6,  # Present instruments
}

# Preset 3: Balanced Mix (neutral starting point)
balanced = {
    "vocal_prominence": 0.05,     # Balanced vocals
    "drum_punch": 0.8,           # Balanced drums
    "bass_foundation": 0.5,      # Balanced bass
    "instrument_presence": 0.5,  # Balanced instruments
}

# Preset 4: Your Issue Fix (loud vocals, weak drums)
fix_balance = {
    "vocal_prominence": 0.01,     # 🎤 Reduce vocal dominance
    "drum_punch": 0.99,           # 🥁 Increase drum punch
    "bass_foundation": 0.6,      # 🎸 Solid foundation
    "instrument_presence": 0.4,  # 🎹 Balanced instruments
}

# 🎯 CHOOSE YOUR BALANCE (uncomment one line)
# selected_balance = vocal_forward
# selected_balance = drum_heavy  
# selected_balance = balanced
selected_balance = fix_balance     # ← This should fix your vocals/drums issue


print("🎚️ Selected Balance Profile:")
for param, value in selected_balance.items():
    param_name = param.replace('_', ' ').title()
    bar = '█' * int(value * 10) + '░' * int((1-value) * 10)
    print(f"  {param_name:18} [{bar}] {value:.1%}")

print("\n💡 How it works:")
print("  • Vocal Prominence: Adjusts presence EQ, compression, reverb send")
print("  • Drum Punch: Enhances transients, compression attack, EQ punch frequencies") 
print("  • Bass Foundation: Controls low-end EQ, compression, mud removal")
print("  • Instrument Presence: Manages clarity EQ, spatial positioning vs vocals")

🎚️ Selected Balance Profile:
  Vocal Prominence   [░░░░░░░░░] 1.0%
  Drum Punch         [█████████] 99.0%
  Bass Foundation    [██████░░░░] 60.0%
  Instrument Presence [████░░░░░░] 40.0%

💡 How it works:
  • Vocal Prominence: Adjusts presence EQ, compression, reverb send
  • Drum Punch: Enhances transients, compression attack, EQ punch frequencies
  • Bass Foundation: Controls low-end EQ, compression, mud removal
  • Instrument Presence: Manages clarity EQ, spatial positioning vs vocals


## 🎚️ Step 3: Manual Volume Balance

Set your preferred channel volumes using the hardcoded dictionary below.

In [5]:
# 🎚️ MANUAL BALANCE CONTROL - SIMPLE HARDCODED DICT

def adjust_volumes(overrides, percent, group=None, exclude_group=None):
    """
    Adjusts volume levels in a channel overrides dict.

    Parameters:
    - overrides (dict): channel -> volume mapping
    - percent (float): percentage change (e.g., +10 for +10%, -20 for -20%)
    - group (str, optional): if given, only adjust this group (prefix match)
    - exclude_group (str, optional): if given, adjust everything except this group
    """
    factor = 1 + percent / 100.0
    new_overrides = {}

    for channel, vol in overrides.items():
        prefix = channel.split('.')[0]  # e.g., "drums", "vocals"

        # Decide whether this channel should be scaled
        if group is not None:
            if prefix == group:
                new_overrides[channel] = vol * factor
            else:
                new_overrides[channel] = vol
        elif exclude_group is not None:
            if prefix != exclude_group:
                new_overrides[channel] = vol * factor
            else:
                new_overrides[channel] = vol
        else:
            # No filter, adjust everything
            new_overrides[channel] = vol * factor

    return new_overrides


channel_overrides = {
    # DRUMS - Much higher (as you prefer)
    'drums.kick': 5,
    'drums.snare': 5,
    'drums.hihat': 2.0,
    'drums.tom': 3.0,       
    'drums.cymbal': 2.0,
    
    # VOCALS - Clear but not overpowering
    'vocals.lead_vocal1': .7,
    'vocals.lead_vocal2': .7,
    'vocals.lead_vocal3': .7,
    
    # BACKING VOCALS - Supporting
    'backvocals.backing_vocal': .3,
    'backvocals.lead_vocal1': .3,
    'backvocals.lead_vocal2': .3,
    'backvocals.lead_vocal3': .3,
    'backvocals.lead_vocal4': 2.0,
    
    # BASS - Solid foundation
    'bass.bass_guitar5': .5,
    'bass.bass1': .5,
    'bass.bass_guitar3': .5,
    'bass.bass_synth2': 4.0,
    'bass.bass_synth4': .5,
    
    # GUITARS - Present but not dominant
    'guitars.electric_guitar2': .5,
    'guitars.electric_guitar3': .3,
    'guitars.electric_guitar4': .5,
    'guitars.electric_guitar5': .5,
    'guitars.electric_guitar6': .5,
    'guitars.acoustic_guitar1': .5,
    
    # KEYS - Background texture
    'keys.bell3': 0.8,
    'keys.clavinet1': 0.5,
    'keys.piano2': 0.8,
    'keys.piano4': 0.8,
    
    # SYNTHS - Atmospheric
    'synths.pad2': 0.8,
    'synths.pad3': 0.4,
    'synths.rythmic_synth1': 0.5,
}
channel_overrides = adjust_volumes(channel_overrides, -30, group=None, exclude_group='drums')
channel_overrides = adjust_volumes(channel_overrides, 50, group=None, exclude_group='guitars')


print("🎚️ Manual balance control applied:")
print(f"  🥁 Drums: 3.0-3.5x (high punch)")
print(f"  🎤 Vocals: 1.5x (clear presence)")
print(f"  🎤 Backing Vocals: 1.0x (supporting)")
print(f"  🎸 Bass: 2.0x (solid foundation)")
print(f"  🎸 Guitars: 1.2x (present)")
print(f"  🎹 Keys/Synths: 0.8-0.9x (texture)")

# Balance parameters for mixing engine
selected_balance = {
    "vocal_prominence": 0.5,    
    "drum_punch": 0.6,          
    "bass_foundation": 0.5,     
    "instrument_presence": 0.5, 
}

🎚️ Manual balance control applied:
  🥁 Drums: 3.0-3.5x (high punch)
  🎤 Vocals: 1.5x (clear presence)
  🎤 Backing Vocals: 1.0x (supporting)
  🎸 Bass: 2.0x (solid foundation)
  🎸 Guitars: 1.2x (present)
  🎹 Keys/Synths: 0.8-0.9x (texture)


## 🔧 Step 3.5: Advanced Low-End Processing

Enhanced bass and drum processing with professional techniques for tight, clean low-end.

In [6]:
# 🔧 ADVANCED BASS & DRUM PROCESSING
# Professional techniques for tight, clean, punchy low-end

def enhance_low_end_processing(session):
    """Apply advanced processing to bass and drums for maximum clarity and punch"""
    
    print("🔧 APPLYING ADVANCED LOW-END PROCESSING")
    print("=" * 50)
    
    # Import advanced DSP functions
    from dsp_premitives import (
        notch_filter, peaking_eq, highpass_filter, lowpass_filter,
        compressor, transient_shaper, shelf_filter
    )
    from audio_utils import db_to_linear
    import numpy as np
    
    processed_channels = 0
    
    # DRUM PROCESSING - Enhanced punch and clarity
    for channel_id, strip in session.channel_strips.items():
        if strip.category.lower() == 'drums':
            print(f"\n🥁 Processing {channel_id}...")
            
            # Kick drum specific processing
            if 'kick' in channel_id.lower():
                # 1. Sub-harmonic enhancement (40-60Hz boost)
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['50hz'] = {'gain': 3.0, 'q': 1.2}  # Sub punch
                
                # 2. Attack enhancement (2-5kHz)
                strip.eq_bands['3khz'] = {'gain': 2.0, 'q': 0.8}  # Click/attack
                
                # 3. Mud removal (200-400Hz notch)
                strip.eq_bands['300hz'] = {'gain': -4.0, 'q': 2.0}  # Remove mud
                
                # 4. Aggressive compression for consistency
                strip.comp_enabled = True
                strip.comp_threshold = -12.0
                strip.comp_ratio = 6.0
                strip.comp_attack = 3.0  # Fast attack for transients
                strip.comp_release = 80.0
                
                print(f"    ✓ Applied kick processing: sub boost, attack enhance, mud cut")
                
            # Snare drum processing
            elif 'snare' in channel_id.lower():
                strip.eq_bands = strip.eq_bands or {}
                
                # 1. Body enhancement (200Hz)
                strip.eq_bands['200hz'] = {'gain': 1.5, 'q': 0.7}  # Body
                
                # 2. Crack/snap (3-5kHz)  
                strip.eq_bands['4khz'] = {'gain': 3.0, 'q': 1.0}  # Snap
                
                # 3. Air and sparkle (10kHz shelf)
                strip.eq_bands['10khz'] = {'gain': 2.0, 'q': 0.5}  # Air
                
                # 4. Parallel compression for body
                strip.comp_enabled = True
                strip.comp_threshold = -8.0
                strip.comp_ratio = 4.0
                strip.comp_attack = 5.0
                strip.comp_release = 100.0
                
                print(f"    ✓ Applied snare processing: body, crack, air enhancement")
                
            # Hi-hat and cymbals
            elif 'hihat' in channel_id.lower() or 'cymbal' in channel_id.lower():
                strip.eq_bands = strip.eq_bands or {}
                
                # 1. Brightness and sparkle
                strip.eq_bands['12khz'] = {'gain': 2.5, 'q': 0.6}  # Sparkle
                
                # 2. Remove harshness
                strip.eq_bands['6khz'] = {'gain': -1.5, 'q': 1.5}  # Reduce harsh
                
                # 3. High-pass filter to remove low-end rumble
                strip.eq_bands['120hz'] = {'gain': -18.0, 'q': 0.7}  # HPF effect
                
                print(f"    ✓ Applied cymbal processing: sparkle, harshness reduction")
            
            processed_channels += 1
    
    # BASS PROCESSING - Tight, clear, powerful
    for channel_id, strip in session.channel_strips.items():
        if strip.category.lower() == 'bass':
            print(f"\n🎸 Processing {channel_id}...")
            
            strip.eq_bands = strip.eq_bands or {}
            
            # Bass guitar processing
            if 'bass_guitar' in channel_id.lower() or 'bass1' in channel_id.lower():
                # 1. Sub-bass foundation (40-80Hz)
                strip.eq_bands['60hz'] = {'gain': 2.0, 'q': 0.8}  # Sub foundation
                
                # 2. Attack and definition (1-2kHz) 
                strip.eq_bands['1500hz'] = {'gain': 2.5, 'q': 1.0}  # Attack/definition
                
                # 3. Mud removal (300-500Hz)
                strip.eq_bands['400hz'] = {'gain': -3.0, 'q': 1.5}  # Remove mud
                
                # 4. Compression for consistency
                strip.comp_enabled = True
                strip.comp_threshold = -15.0
                strip.comp_ratio = 3.0
                strip.comp_attack = 8.0
                strip.comp_release = 120.0
                
                print(f"    ✓ Applied bass guitar processing: sub foundation, attack, mud removal")
                
            # Bass synth processing  
            elif 'bass_synth' in channel_id.lower():
                # 1. Deep sub enhancement
                strip.eq_bands['40hz'] = {'gain': 3.0, 'q': 1.0}  # Deep sub
                
                # 2. Harmonic clarity (800Hz-2kHz)
                strip.eq_bands['1200hz'] = {'gain': 1.5, 'q': 0.8}  # Harmonics
                
                # 3. Clean up mid-muck
                strip.eq_bands['350hz'] = {'gain': -2.5, 'q': 2.0}  # Clean mids
                
                # 4. Gentle compression to maintain dynamics
                strip.comp_enabled = True
                strip.comp_threshold = -18.0
                strip.comp_ratio = 2.5
                strip.comp_attack = 12.0
                strip.comp_release = 150.0
                
                print(f"    ✓ Applied bass synth processing: deep sub, harmonics, cleanup")
                
            processed_channels += 1
    
    # BUS-LEVEL PROCESSING - Glue and cohesion
    print(f"\n🎛️ Applying bus-level processing...")
    
    # Enhanced drum bus processing
    if 'drum_bus' in session.buses:
        drum_bus = session.buses['drum_bus']
        
        # More aggressive glue compression
        drum_bus.comp_enabled = True
        drum_bus.comp_threshold = -8.0
        drum_bus.comp_ratio = 3.0
        drum_bus.glue_amount = 0.7  # Strong glue
        
        # Subtle harmonic saturation for punch
        drum_bus.saturation = 0.15
        
        print(f"    ✓ Drum bus: glue compression (3:1), harmonic saturation")
    
    # Enhanced bass bus processing  
    if 'bass_bus' in session.buses:
        bass_bus = session.buses['bass_bus']
        
        # Gentle bus compression for consistency
        bass_bus.comp_enabled = True
        bass_bus.comp_threshold = -12.0
        bass_bus.comp_ratio = 2.0
        bass_bus.glue_amount = 0.4
        
        # Light saturation for warmth and harmonics
        bass_bus.saturation = 0.1
        
        print(f"    ✓ Bass bus: gentle compression (2:1), harmonic warmth")
    
    print(f"\n✅ Enhanced processing applied to {processed_channels} channels")
    print(f"🔧 Techniques used:")
    print(f"   • Surgical EQ (notching, boosting, mud removal)")
    print(f"   • Multi-band compression for consistency")
    print(f"   • Harmonic saturation for punch and warmth")  
    print(f"   • Bus-level glue compression")
    print(f"   • Transient enhancement for attack clarity")

# Apply the enhanced processing
if 'session' in locals():
    enhance_low_end_processing(session)
    print("\n🎯 Low-end processing complete! Ready for enhanced mix.")
else:
    print("❌ No session found - please run the previous cells first.")

❌ No session found - please run the previous cells first.


## 🎛️ Step 3.6: Advanced Dynamics & Spatial Processing

Professional techniques for punch, cohesion, and spatial clarity in the low-end.

In [7]:
# 🎛️ ADVANCED DYNAMICS & SPATIAL PROCESSING
# Professional techniques for punch, cohesion, and spatial clarity

def apply_advanced_dynamics_and_spatial(session):
    """Apply pro-level dynamics, sidechain, parallel compression, and spatial processing"""
    
    print("🎛️ APPLYING ADVANCED DYNAMICS & SPATIAL PROCESSING")
    print("=" * 60)
    
    from dsp_premitives import (
        compressor, transient_shaper, mid_side_encode, mid_side_decode,
        stereo_widener, highpass_filter, lowpass_filter
    )
    from audio_utils import db_to_linear, ensure_stereo, to_mono
    import numpy as np
    
    # STEP 1: SIDECHAIN COMPRESSION - Duck bass when kick hits
    print("\n🔗 SIDECHAIN COMPRESSION")
    print("-" * 30)
    
    # Find kick and bass channels for sidechain setup
    kick_channels = [ch_id for ch_id in session.channel_strips.keys() if 'kick' in ch_id.lower()]
    bass_channels = [ch_id for ch_id in session.channel_strips.keys() if session.channel_strips[ch_id].category.lower() == 'bass']
    
    if kick_channels and bass_channels:
        # Simulate sidechain by adjusting bass compression when kick is present
        for bass_id in bass_channels:
            bass_strip = session.channel_strips[bass_id]
            
            # Enhanced compression with sidechain-style settings
            bass_strip.comp_enabled = True
            bass_strip.comp_threshold = -20.0  # Lower threshold for more ducking
            bass_strip.comp_ratio = 4.0       # Higher ratio for pronounced effect
            bass_strip.comp_attack = 1.0      # Very fast attack to catch kick transients
            bass_strip.comp_release = 150.0   # Medium release to pump with kick rhythm
            
            print(f"    ✓ {bass_id}: Sidechain-style compression (4:1, 1ms attack)")
    
    # STEP 2: PARALLEL COMPRESSION (New York Style)
    print("\n🏙️ PARALLEL COMPRESSION (NEW YORK STYLE)")
    print("-" * 40)
    
    # Apply parallel compression to drums and bass
    rhythm_channels = [ch_id for ch_id in session.channel_strips.keys() 
                      if session.channel_strips[ch_id].category.lower() in ['drums', 'bass']]
    
    for ch_id in rhythm_channels:
        strip = session.channel_strips[ch_id]
        
        # Create "parallel" effect by blending compressed and uncompressed signals
        # We simulate this by using gentler compression with makeup gain
        if strip.category.lower() == 'drums':
            # Heavy compression settings for parallel blend
            strip.comp_enabled = True
            strip.comp_threshold = -25.0   # Very low threshold
            strip.comp_ratio = 8.0         # Heavy compression
            strip.comp_attack = 0.5        # Super fast
            strip.comp_release = 50.0      # Quick release
            
            # Compensate with gain (simulating parallel blend)
            strip.gain *= 1.15  # Boost to simulate parallel blend
            
            print(f"    ✓ {ch_id}: NY compression (8:1, aggressive settings)")
        
        elif strip.category.lower() == 'bass':
            # Gentler parallel compression for bass
            strip.comp_threshold = -18.0
            strip.comp_ratio = 6.0
            strip.comp_attack = 2.0
            strip.comp_release = 80.0
            strip.gain *= 1.1
            
            print(f"    ✓ {ch_id}: Bass parallel compression (6:1)")
    
    # STEP 3: TRANSIENT SHAPING
    print("\n⚡ TRANSIENT SHAPING")
    print("-" * 25)
    
    for ch_id, strip in session.channel_strips.items():
        if strip.category.lower() == 'drums':
            # Enhance drum transients through EQ and compression timing
            if 'kick' in ch_id.lower():
                # Kick: Enhance attack, control sustain
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['2500hz'] = {'gain': 2.5, 'q': 1.2}  # Attack clarity
                strip.comp_attack = 2.0   # Allow initial transient through
                print(f"    ✓ {ch_id}: Enhanced attack transients")
                
            elif 'snare' in ch_id.lower():
                # Snare: Maximum crack and snap
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['5000hz'] = {'gain': 3.0, 'q': 0.8}  # Crack frequency
                strip.comp_attack = 3.0   # Preserve snap
                print(f"    ✓ {ch_id}: Enhanced crack and snap")
        
        elif strip.category.lower() == 'bass':
            # Bass: Control sustain, enhance note definition
            if 'bass_guitar' in ch_id.lower():
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['2000hz'] = {'gain': 1.8, 'q': 1.0}  # Note definition
                strip.comp_attack = 8.0   # Slower attack to preserve note start
                print(f"    ✓ {ch_id}: Enhanced note definition")
    
    # STEP 4: ADVANCED GLUE COMPRESSION
    print("\n🧲 ADVANCED GLUE COMPRESSION")
    print("-" * 35)
    
    # Enhanced bus processing for ultimate cohesion
    if 'drum_bus' in session.buses:
        drum_bus = session.buses['drum_bus']
        drum_bus.comp_enabled = True
        drum_bus.comp_threshold = -6.0   # Higher threshold for glue only
        drum_bus.comp_ratio = 2.5        # Gentle ratio for musicality
        drum_bus.glue_amount = 0.8       # Strong glue effect
        drum_bus.saturation = 0.2        # More saturation for character
        print(f"    ✓ Drum bus: Ultimate glue compression (2.5:1) + saturation")
    
    if 'bass_bus' in session.buses:
        bass_bus = session.buses['bass_bus']
        bass_bus.comp_enabled = True
        bass_bus.comp_threshold = -10.0
        bass_bus.comp_ratio = 2.0
        bass_bus.glue_amount = 0.5
        bass_bus.saturation = 0.15       # Harmonic enhancement
        print(f"    ✓ Bass bus: Cohesive glue compression (2:1) + warmth")
    
    # STEP 5: PHASE & SPATIAL PROCESSING
    print("\n🌊 PHASE & SPATIAL PROCESSING")
    print("-" * 35)
    
    # Mono compatibility for low frequencies
    for ch_id, strip in session.channel_strips.items():
        if strip.category.lower() == 'bass':
            # Force bass content below 120Hz to mono (simulated with pan center)
            strip.pan = 0.0  # Center panning for mono compatibility
            
            # Add subtle stereo width to upper harmonics only (simulated with EQ)
            strip.eq_bands = strip.eq_bands or {}
            strip.eq_bands['800hz'] = {'gain': 0.8, 'q': 0.6}  # Slight boost for width perception
            
            print(f"    ✓ {ch_id}: Mono low-end, subtle harmonic width")
    
    # Enhanced stereo processing for drums
    for ch_id, strip in session.channel_strips.items():
        if strip.category.lower() == 'drums':
            if 'kick' in ch_id.lower():
                # Kick stays centered for power
                strip.pan = 0.0
                print(f"    ✓ {ch_id}: Centered for maximum power")
            elif 'snare' in ch_id.lower():
                # Snare slightly off-center for width
                strip.pan = 0.1  # Subtle right
                print(f"    ✓ {ch_id}: Subtle positioning for width")
    
    # STEP 6: SATURATION & HARMONIC ENHANCEMENT
    print("\n🔥 SATURATION & HARMONIC ENHANCEMENT")
    print("-" * 45)
    
    saturation_applied = 0
    
    for ch_id, strip in session.channel_strips.items():
        if strip.category.lower() in ['drums', 'bass']:
            # Apply harmonic EQ boosts to simulate exciter/saturation
            strip.eq_bands = strip.eq_bands or {}
            
            if strip.category.lower() == 'bass':
                # Bass: Add upper harmonics for small speaker translation
                strip.eq_bands['1600hz'] = {'gain': 1.2, 'q': 0.7}  # 2nd harmonic region
                strip.eq_bands['3200hz'] = {'gain': 0.8, 'q': 0.8}  # 4th harmonic region
                print(f"    ✓ {ch_id}: Harmonic enhancement for small speakers")
                
            elif strip.category.lower() == 'drums':
                # Drums: Add character and punch through harmonic emphasis
                if 'kick' in ch_id.lower():
                    strip.eq_bands['80hz'] = {'gain': 1.0, 'q': 1.0}   # Fundamental warmth
                    strip.eq_bands['160hz'] = {'gain': 0.6, 'q': 0.8}  # 2nd harmonic
                elif 'snare' in ch_id.lower():
                    strip.eq_bands['240hz'] = {'gain': 0.5, 'q': 0.7}  # Body harmonic
                    
                print(f"    ✓ {ch_id}: Harmonic character enhancement")
            
            saturation_applied += 1
    
    # STEP 7: HEADROOM & BALANCE OPTIMIZATION
    print("\n📊 HEADROOM & BALANCE OPTIMIZATION")
    print("-" * 40)
    
    # Optimize levels for maximum headroom while maintaining punch
    total_adjustments = 0
    
    for ch_id, strip in session.channel_strips.items():
        if strip.category.lower() in ['drums', 'bass']:
            original_gain = strip.gain
            
            # Smart gain staging: louder drums, controlled bass
            if strip.category.lower() == 'drums':
                if 'kick' in ch_id.lower() or 'snare' in ch_id.lower():
                    # Keep punch elements loud but controlled
                    strip.gain = min(strip.gain, 2.8)  # Cap to prevent clipping
                else:
                    # Other drums slightly lower for headroom
                    strip.gain *= 0.95
                    
            elif strip.category.lower() == 'bass':
                # Bass: Solid but not overpowering
                strip.gain *= 0.9  # Slight reduction for headroom
            
            if abs(strip.gain - original_gain) > 0.01:
                total_adjustments += 1
    
    print(f"    ✓ Optimized {total_adjustments} channels for headroom")
    print(f"    ✓ Maintained punch while preserving dynamics")
    
    # FINAL SUMMARY
    print(f"\n" + "=" * 60)
    print(f"✅ ADVANCED PROCESSING COMPLETE!")
    print(f"=" * 60)
    print(f"🔗 Sidechain: Bass ducks for kick clarity")
    print(f"🏙️ Parallel compression: Added punch and body") 
    print(f"⚡ Transient shaping: Enhanced attack/sustain balance")
    print(f"🧲 Glue compression: Unified rhythm section")
    print(f"🌊 Phase/Spatial: Mono low-end, strategic positioning")
    print(f"🔥 Saturation: Harmonic enhancement for translation")
    print(f"📊 Balance: Optimized headroom with maintained punch")
    print(f"\n🎯 Your mix now has professional-level dynamics and spatial clarity!")

# Apply the advanced processing
if 'session' in locals():
    apply_advanced_dynamics_and_spatial(session)
    print("\n🚀 Ready to process the enhanced mix!")
else:
    print("❌ No session found - please run the previous cells first.")

❌ No session found - please run the previous cells first.


# ✨ MIDRANGE & HIGH-FREQUENCY MASTERY
# Professional processing for clarity, presence, and air

def apply_midrange_and_high_freq_processing(session):
    """Apply pro-level midrange clarity and high-frequency air/sparkle processing"""
    
    print("✨ APPLYING MIDRANGE & HIGH-FREQUENCY MASTERY")
    print("=" * 55)
    
    from dsp_premitives import (
        peaking_eq, shelf_filter, notch_filter, highpass_filter,
        compressor, mid_side_encode, mid_side_decode
    )
    from audio_utils import db_to_linear
    import numpy as np
    
    # MIDRANGE PROCESSING (250Hz - 4kHz)
    print("\n🎯 MIDRANGE PROCESSING (250Hz - 4kHz)")
    print("-" * 45)
    
    # Step 1: ANTI-MUD & BOXINESS REMOVAL
    print("\n🧹 Anti-Mud & Boxiness Removal")
    print("-" * 35)
    
    mud_removed = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if strip.category.lower() == 'guitars':
            # Guitar mud and boxiness removal
            strip.eq_bands['350hz'] = {'gain': -2.5, 'q': 1.8}  # Remove mud
            strip.eq_bands['450hz'] = {'gain': -1.5, 'q': 2.2}  # Remove boxiness
            print(f"    ✓ {ch_id}: Guitar mud removal (350Hz) + boxiness cut (450Hz)")
            mud_removed += 1
            
        elif strip.category.lower() == 'keys':
            # Keys mud and conflict removal
            strip.eq_bands['320hz'] = {'gain': -2.0, 'q': 1.5}  # Clean mids
            strip.eq_bands['480hz'] = {'gain': -1.8, 'q': 1.8}  # Remove boxy resonance
            print(f"    ✓ {ch_id}: Keys mid cleanup (320Hz, 480Hz)")
            mud_removed += 1
            
        elif strip.category.lower() == 'synths':
            # Synth clarity and space making
            if 'pad' in ch_id.lower():
                strip.eq_bands['400hz'] = {'gain': -3.0, 'q': 1.2}  # Make space for vocals
                print(f"    ✓ {ch_id}: Pad mud removal for vocal space")
                mud_removed += 1
    
    # Step 2: VOCAL PRESENCE & CLARITY
    print("\n🎤 Vocal Presence & Clarity")  
    print("-" * 30)
    
    vocals_processed = 0
    for ch_id, strip in session.channel_strips.items():
        if 'vocal' in strip.category.lower():
            strip.eq_bands = strip.eq_bands or {}
            
            if 'lead_vocal' in ch_id.lower() or 'vocals' in strip.category.lower():
                # Lead vocal presence and clarity
                strip.eq_bands['300hz'] = {'gain': -1.0, 'q': 0.8}   # Clean body
                strip.eq_bands['2500hz'] = {'gain': 2.5, 'q': 1.2}  # Presence boost
                strip.eq_bands['3500hz'] = {'gain': 1.8, 'q': 0.9}  # Clarity
                
                # Advanced compression for vocal consistency
                strip.comp_enabled = True
                strip.comp_threshold = -18.0
                strip.comp_ratio = 3.5
                strip.comp_attack = 8.0   # Preserve natural attack
                strip.comp_release = 120.0
                
                print(f"    ✓ {ch_id}: Lead vocal presence (2.5kHz) + clarity (3.5kHz)")
                
            elif 'backing' in ch_id.lower() or 'backvocals' in strip.category.lower():
                # Backing vocals - make space for lead
                strip.eq_bands['2200hz'] = {'gain': -1.2, 'q': 1.0}  # Make space for lead
                strip.eq_bands['3800hz'] = {'gain': 1.2, 'q': 0.8}   # Backing sparkle
                
                print(f"    ✓ {ch_id}: Backing vocal space-making + subtle sparkle")
            
            vocals_processed += 1
    
    # Step 3: INSTRUMENTAL PRESENCE & SEPARATION
    print("\n🎸 Instrumental Presence & Separation")
    print("-" * 40)
    
    instruments_processed = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if strip.category.lower() == 'guitars':
            if 'electric' in ch_id.lower():
                # Electric guitar presence without vocal conflict
                strip.eq_bands['1800hz'] = {'gain': 1.5, 'q': 0.9}  # Note definition
                strip.eq_bands['2800hz'] = {'gain': -0.8, 'q': 1.5} # Make space for vocals
                strip.eq_bands['4200hz'] = {'gain': 1.2, 'q': 0.7}  # Harmonics
                print(f"    ✓ {ch_id}: Electric guitar presence with vocal space")
                
            elif 'acoustic' in ch_id.lower():
                # Acoustic guitar body and sparkle
                strip.eq_bands['280hz'] = {'gain': 1.0, 'q': 0.8}   # Body
                strip.eq_bands['2000hz'] = {'gain': 1.8, 'q': 1.0}  # Fingering clarity
                strip.eq_bands['3600hz'] = {'gain': 0.8, 'q': 0.6}  # String brightness
                print(f"    ✓ {ch_id}: Acoustic guitar body + fingering clarity")
                
            instruments_processed += 1
            
        elif strip.category.lower() == 'keys':
            # Keys presence and character
            strip.eq_bands['800hz'] = {'gain': 0.8, 'q': 0.7}   # Body/warmth
            strip.eq_bands['2200hz'] = {'gain': -1.0, 'q': 1.2} # Make vocal space
            strip.eq_bands['3800hz'] = {'gain': 1.0, 'q': 0.8}  # Note attack
            print(f"    ✓ {ch_id}: Keys warmth + attack with vocal space")
            instruments_processed += 1
    
    # HIGH-FREQUENCY PROCESSING (4kHz - 20kHz)
    print(f"\n🌟 HIGH-FREQUENCY PROCESSING (4kHz - 20kHz)")
    print("-" * 48)
    
    # Step 4: DE-ESSING & HARSHNESS CONTROL
    print("\n🤫 De-essing & Harshness Control")
    print("-" * 35)
    
    harshness_controlled = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if 'vocal' in strip.category.lower():
            # Vocal sibilance and harshness control
            strip.eq_bands['6500hz'] = {'gain': -2.0, 'q': 2.5}  # De-ess frequency
            strip.eq_bands['4800hz'] = {'gain': -1.2, 'q': 1.8}  # Harshness control
            print(f"    ✓ {ch_id}: Vocal de-essing (6.5kHz) + harshness control")
            harshness_controlled += 1
            
        elif strip.category.lower() == 'drums':
            if 'snare' in ch_id.lower():
                # Snare harshness control while keeping crack
                strip.eq_bands['3200hz'] = {'gain': -1.5, 'q': 2.0}  # Reduce harshness
                strip.eq_bands['8500hz'] = {'gain': 1.5, 'q': 0.8}   # Keep crack/snap
                print(f"    ✓ {ch_id}: Snare harshness control + crack enhancement")
                harshness_controlled += 1
                
            elif 'cymbal' in ch_id.lower() or 'hihat' in ch_id.lower():
                # Cymbal/hihat harshness control
                strip.eq_bands['5500hz'] = {'gain': -2.5, 'q': 1.8}  # Harsh resonance
                strip.eq_bands['11000hz'] = {'gain': 1.8, 'q': 0.6}  # Sparkle above harshness
                print(f"    ✓ {ch_id}: Cymbal harshness control + sparkle")
                harshness_controlled += 1
    
    # Step 5: AIR & SPARKLE ENHANCEMENT
    print("\n✨ Air & Sparkle Enhancement")
    print("-" * 30)
    
    air_enhanced = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if 'vocal' in strip.category.lower():
            # Vocal air and presence
            strip.eq_bands['12000hz'] = {'gain': 1.5, 'q': 0.5}  # Air frequency
            strip.eq_bands['16000hz'] = {'gain': 0.8, 'q': 0.4}  # Ultra-high sparkle
            print(f"    ✓ {ch_id}: Vocal air (12kHz) + ultra-high sparkle")
            air_enhanced += 1
            
        elif strip.category.lower() == 'drums':
            if 'cymbal' in ch_id.lower() or 'hihat' in ch_id.lower():
                # Cymbal/hihat air and shimmer
                strip.eq_bands['14000hz'] = {'gain': 2.2, 'q': 0.6}  # Shimmer
                strip.eq_bands['18000hz'] = {'gain': 1.0, 'q': 0.3}  # Ultra-air
                print(f"    ✓ {ch_id}: Cymbal shimmer + ultra-air")
                air_enhanced += 1
                
        elif strip.category.lower() == 'guitars':
            if 'acoustic' in ch_id.lower():
                # Acoustic guitar air and string detail
                strip.eq_bands['10000hz'] = {'gain': 1.5, 'q': 0.7}  # String detail
                strip.eq_bands['15000hz'] = {'gain': 0.8, 'q': 0.4}  # Air
                print(f"    ✓ {ch_id}: Acoustic guitar string detail + air")
                air_enhanced += 1
    
    # Step 6: STEREO IMAGING & SPATIAL ENHANCEMENT
    print("\n🌐 Stereo Imaging & Spatial Enhancement")
    print("-" * 42)
    
    spatial_enhanced = 0
    for ch_id, strip in session.channel_strips.items():
        
        if strip.category.lower() == 'guitars':
            # Guitar stereo positioning
            if 'electric_guitar2' in ch_id.lower():
                strip.pan = -0.4  # Left
            elif 'electric_guitar3' in ch_id.lower():
                strip.pan = 0.4   # Right
            elif 'electric_guitar4' in ch_id.lower():
                strip.pan = -0.6  # Hard left
            elif 'electric_guitar5' in ch_id.lower():
                strip.pan = 0.6   # Hard right
            print(f"    ✓ {ch_id}: Stereo positioned for width")
            spatial_enhanced += 1
            
        elif 'backing' in ch_id.lower() or ('backvocals' in strip.category.lower() and 'lead_vocal' in ch_id.lower()):
            # Backing vocal spread
            if 'lead_vocal1' in ch_id.lower():
                strip.pan = -0.3
            elif 'lead_vocal2' in ch_id.lower():
                strip.pan = 0.3
            elif 'lead_vocal3' in ch_id.lower():
                strip.pan = -0.5
            elif 'lead_vocal4' in ch_id.lower():
                strip.pan = 0.5
            print(f"    ✓ {ch_id}: Backing vocal stereo spread")
            spatial_enhanced += 1
    
    # Step 7: ADVANCED HARMONIC ENHANCEMENT
    print("\n🎵 Advanced Harmonic Enhancement")
    print("-" * 35)
    
    harmonics_enhanced = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        # Add subtle harmonic series enhancement for all midrange instruments
        if strip.category.lower() in ['guitars', 'keys', 'synths']:
            # 2nd harmonic region enhancement
            strip.eq_bands['1400hz'] = {'gain': 0.5, 'q': 0.6}  # 2nd harmonic warmth
            strip.eq_bands['2800hz'] = {'gain': 0.3, 'q': 0.7}  # 4th harmonic presence
            strip.eq_bands['5600hz'] = {'gain': 0.2, 'q': 0.8}  # 8th harmonic sparkle
            
            print(f"    ✓ {ch_id}: Harmonic series enhancement (2nd, 4th, 8th)")
            harmonics_enhanced += 1
    
    # BUS-LEVEL MIDRANGE & HIGH-FREQ PROCESSING
    print("\n🎛️ Bus-Level Processing")
    print("-" * 25)
    
    # Enhanced vocal bus processing
    if 'vocal_bus' in session.buses:
        vocal_bus = session.buses['vocal_bus']
        
        # Vocal bus presence and air
        vocal_bus.comp_enabled = True
        vocal_bus.comp_threshold = -12.0
        vocal_bus.comp_ratio = 2.5
        vocal_bus.glue_amount = 0.3  # Light glue for natural vocals
        
        print(f"    ✓ Vocal bus: Light glue compression + presence enhancement")
    
    # Enhanced instrument bus processing
    if 'instrument_bus' in session.buses:
        instrument_bus = session.buses['instrument_bus']
        
        # Instrument bus width and air
        instrument_bus.width = 1.3  # Wider for spaciousness
        instrument_bus.saturation = 0.05  # Very light harmonic enhancement
        
        print(f"    ✓ Instrument bus: Enhanced width (130%) + harmonic warmth")
    
    # FINAL SUMMARY
    print(f"\n" + "=" * 55)
    print(f"✅ MIDRANGE & HIGH-FREQUENCY MASTERY COMPLETE!")
    print(f"=" * 55)
    print(f"🧹 Mud removal: {mud_removed} channels cleaned")
    print(f"🎤 Vocal processing: {vocals_processed} channels enhanced")
    print(f"🎸 Instruments: {instruments_processed} channels optimized") 
    print(f"🤫 Harshness control: {harshness_controlled} channels tamed")
    print(f"✨ Air enhancement: {air_enhanced} channels sparkled")
    print(f"🌐 Stereo imaging: {spatial_enhanced} channels positioned")
    print(f"🎵 Harmonic enhancement: {harmonics_enhanced} channels enriched")
    print(f"\n🎯 Your mix now has professional clarity, presence, and air!")
    print(f"💫 Ready for final mastering with full-spectrum enhancement!")

# Apply the midrange and high-frequency processing
if 'session' in locals():
    apply_midrange_and_high_freq_processing(session)
    print("\n🚀 Full-spectrum professional processing complete!")
else:
    print("❌ No session found - please run the previous cells first.")

In [8]:
# ✨ MIDRANGE & HIGH-FREQUENCY MASTERY
# Professional processing for clarity, presence, and air

def apply_midrange_and_high_freq_processing(session):
    """Apply pro-level midrange clarity and high-frequency air/sparkle processing"""
    
    print("✨ APPLYING MIDRANGE & HIGH-FREQUENCY MASTERY")
    print("=" * 55)
    
    from dsp_premitives import (
        peaking_eq, shelf_filter, notch_filter, highpass_filter,
        compressor, mid_side_encode, mid_side_decode
    )
    from audio_utils import db_to_linear
    import numpy as np
    
    # MIDRANGE PROCESSING (250Hz - 4kHz)
    print("\n🎯 MIDRANGE PROCESSING (250Hz - 4kHz)")
    print("-" * 45)
    
    # Step 1: ANTI-MUD & BOXINESS REMOVAL
    print("\n🧹 Anti-Mud & Boxiness Removal")
    print("-" * 35)
    
    mud_removed = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if strip.category.lower() == 'guitars':
            # Guitar mud and boxiness removal
            strip.eq_bands['350hz'] = {'gain': -2.5, 'q': 1.8}  # Remove mud
            strip.eq_bands['450hz'] = {'gain': -1.5, 'q': 2.2}  # Remove boxiness
            print(f"    ✓ {ch_id}: Guitar mud removal (350Hz) + boxiness cut (450Hz)")
            mud_removed += 1
            
        elif strip.category.lower() == 'keys':
            # Keys mud and conflict removal
            strip.eq_bands['320hz'] = {'gain': -2.0, 'q': 1.5}  # Clean mids
            strip.eq_bands['480hz'] = {'gain': -1.8, 'q': 1.8}  # Remove boxy resonance
            print(f"    ✓ {ch_id}: Keys mid cleanup (320Hz, 480Hz)")
            mud_removed += 1
            
        elif strip.category.lower() == 'synths':
            # Synth clarity and space making
            if 'pad' in ch_id.lower():
                strip.eq_bands['400hz'] = {'gain': -3.0, 'q': 1.2}  # Make space for vocals
                print(f"    ✓ {ch_id}: Pad mud removal for vocal space")
                mud_removed += 1
    
    # Step 2: VOCAL PRESENCE & CLARITY
    print("\n🎤 Vocal Presence & Clarity")  
    print("-" * 30)
    
    vocals_processed = 0
    for ch_id, strip in session.channel_strips.items():
        if 'vocal' in strip.category.lower():
            strip.eq_bands = strip.eq_bands or {}
            
            if 'lead_vocal' in ch_id.lower() or 'vocals' in strip.category.lower():
                # Lead vocal presence and clarity
                strip.eq_bands['300hz'] = {'gain': -1.0, 'q': 0.8}   # Clean body
                strip.eq_bands['2500hz'] = {'gain': 2.5, 'q': 1.2}  # Presence boost
                strip.eq_bands['3500hz'] = {'gain': 1.8, 'q': 0.9}  # Clarity
                
                # Advanced compression for vocal consistency
                strip.comp_enabled = True
                strip.comp_threshold = -18.0
                strip.comp_ratio = 3.5
                strip.comp_attack = 8.0   # Preserve natural attack
                strip.comp_release = 120.0
                
                print(f"    ✓ {ch_id}: Lead vocal presence (2.5kHz) + clarity (3.5kHz)")
                
            elif 'backing' in ch_id.lower() or 'backvocals' in strip.category.lower():
                # Backing vocals - make space for lead
                strip.eq_bands['2200hz'] = {'gain': -1.2, 'q': 1.0}  # Make space for lead
                strip.eq_bands['3800hz'] = {'gain': 1.2, 'q': 0.8}   # Backing sparkle
                
                print(f"    ✓ {ch_id}: Backing vocal space-making + subtle sparkle")
            
            vocals_processed += 1
    
    # Step 3: INSTRUMENTAL PRESENCE & SEPARATION
    print("\n🎸 Instrumental Presence & Separation")
    print("-" * 40)
    
    instruments_processed = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if strip.category.lower() == 'guitars':
            if 'electric' in ch_id.lower():
                # Electric guitar presence without vocal conflict
                strip.eq_bands['1800hz'] = {'gain': 1.5, 'q': 0.9}  # Note definition
                strip.eq_bands['2800hz'] = {'gain': -0.8, 'q': 1.5} # Make space for vocals
                strip.eq_bands['4200hz'] = {'gain': 1.2, 'q': 0.7}  # Harmonics
                print(f"    ✓ {ch_id}: Electric guitar presence with vocal space")
                
            elif 'acoustic' in ch_id.lower():
                # Acoustic guitar body and sparkle
                strip.eq_bands['280hz'] = {'gain': 1.0, 'q': 0.8}   # Body
                strip.eq_bands['2000hz'] = {'gain': 1.8, 'q': 1.0}  # Fingering clarity
                strip.eq_bands['3600hz'] = {'gain': 0.8, 'q': 0.6}  # String brightness
                print(f"    ✓ {ch_id}: Acoustic guitar body + fingering clarity")
                
            instruments_processed += 1
            
        elif strip.category.lower() == 'keys':
            # Keys presence and character
            strip.eq_bands['800hz'] = {'gain': 0.8, 'q': 0.7}   # Body/warmth
            strip.eq_bands['2200hz'] = {'gain': -1.0, 'q': 1.2} # Make vocal space
            strip.eq_bands['3800hz'] = {'gain': 1.0, 'q': 0.8}  # Note attack
            print(f"    ✓ {ch_id}: Keys warmth + attack with vocal space")
            instruments_processed += 1
    
    # HIGH-FREQUENCY PROCESSING (4kHz - 20kHz)
    print(f"\n🌟 HIGH-FREQUENCY PROCESSING (4kHz - 20kHz)")
    print("-" * 48)
    
    # Step 4: DE-ESSING & HARSHNESS CONTROL
    print("\n🤫 De-essing & Harshness Control")
    print("-" * 35)
    
    harshness_controlled = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if 'vocal' in strip.category.lower():
            # Vocal sibilance and harshness control
            strip.eq_bands['6500hz'] = {'gain': -2.0, 'q': 2.5}  # De-ess frequency
            strip.eq_bands['4800hz'] = {'gain': -1.2, 'q': 1.8}  # Harshness control
            print(f"    ✓ {ch_id}: Vocal de-essing (6.5kHz) + harshness control")
            harshness_controlled += 1
            
        elif strip.category.lower() == 'drums':
            if 'snare' in ch_id.lower():
                # Snare harshness control while keeping crack
                strip.eq_bands['3200hz'] = {'gain': -1.5, 'q': 2.0}  # Reduce harshness
                strip.eq_bands['8500hz'] = {'gain': 1.5, 'q': 0.8}   # Keep crack/snap
                print(f"    ✓ {ch_id}: Snare harshness control + crack enhancement")
                harshness_controlled += 1
                
            elif 'cymbal' in ch_id.lower() or 'hihat' in ch_id.lower():
                # Cymbal/hihat harshness control
                strip.eq_bands['5500hz'] = {'gain': -2.5, 'q': 1.8}  # Harsh resonance
                strip.eq_bands['11000hz'] = {'gain': 1.8, 'q': 0.6}  # Sparkle above harshness
                print(f"    ✓ {ch_id}: Cymbal harshness control + sparkle")
                harshness_controlled += 1
    
    # Step 5: AIR & SPARKLE ENHANCEMENT
    print("\n✨ Air & Sparkle Enhancement")
    print("-" * 30)
    
    air_enhanced = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        if 'vocal' in strip.category.lower():
            # Vocal air and presence
            strip.eq_bands['12000hz'] = {'gain': 1.5, 'q': 0.5}  # Air frequency
            strip.eq_bands['16000hz'] = {'gain': 0.8, 'q': 0.4}  # Ultra-high sparkle
            print(f"    ✓ {ch_id}: Vocal air (12kHz) + ultra-high sparkle")
            air_enhanced += 1
            
        elif strip.category.lower() == 'drums':
            if 'cymbal' in ch_id.lower() or 'hihat' in ch_id.lower():
                # Cymbal/hihat air and shimmer
                strip.eq_bands['14000hz'] = {'gain': 2.2, 'q': 0.6}  # Shimmer
                strip.eq_bands['18000hz'] = {'gain': 1.0, 'q': 0.3}  # Ultra-air
                print(f"    ✓ {ch_id}: Cymbal shimmer + ultra-air")
                air_enhanced += 1
                
        elif strip.category.lower() == 'guitars':
            if 'acoustic' in ch_id.lower():
                # Acoustic guitar air and string detail
                strip.eq_bands['10000hz'] = {'gain': 1.5, 'q': 0.7}  # String detail
                strip.eq_bands['15000hz'] = {'gain': 0.8, 'q': 0.4}  # Air
                print(f"    ✓ {ch_id}: Acoustic guitar string detail + air")
                air_enhanced += 1
    
    # Step 6: STEREO IMAGING & SPATIAL ENHANCEMENT
    print("\n🌐 Stereo Imaging & Spatial Enhancement")
    print("-" * 42)
    
    spatial_enhanced = 0
    for ch_id, strip in session.channel_strips.items():
        
        if strip.category.lower() == 'guitars':
            # Guitar stereo positioning
            if 'electric_guitar2' in ch_id.lower():
                strip.pan = -0.4  # Left
            elif 'electric_guitar3' in ch_id.lower():
                strip.pan = 0.4   # Right
            elif 'electric_guitar4' in ch_id.lower():
                strip.pan = -0.6  # Hard left
            elif 'electric_guitar5' in ch_id.lower():
                strip.pan = 0.6   # Hard right
            print(f"    ✓ {ch_id}: Stereo positioned for width")
            spatial_enhanced += 1
            
        elif 'backing' in ch_id.lower() or ('backvocals' in strip.category.lower() and 'lead_vocal' in ch_id.lower()):
            # Backing vocal spread
            if 'lead_vocal1' in ch_id.lower():
                strip.pan = -0.3
            elif 'lead_vocal2' in ch_id.lower():
                strip.pan = 0.3
            elif 'lead_vocal3' in ch_id.lower():
                strip.pan = -0.5
            elif 'lead_vocal4' in ch_id.lower():
                strip.pan = 0.5
            print(f"    ✓ {ch_id}: Backing vocal stereo spread")
            spatial_enhanced += 1
    
    # Step 7: ADVANCED HARMONIC ENHANCEMENT
    print("\n🎵 Advanced Harmonic Enhancement")
    print("-" * 35)
    
    harmonics_enhanced = 0
    for ch_id, strip in session.channel_strips.items():
        strip.eq_bands = strip.eq_bands or {}
        
        # Add subtle harmonic series enhancement for all midrange instruments
        if strip.category.lower() in ['guitars', 'keys', 'synths']:
            # 2nd harmonic region enhancement
            strip.eq_bands['1400hz'] = {'gain': 0.5, 'q': 0.6}  # 2nd harmonic warmth
            strip.eq_bands['2800hz'] = {'gain': 0.3, 'q': 0.7}  # 4th harmonic presence
            strip.eq_bands['5600hz'] = {'gain': 0.2, 'q': 0.8}  # 8th harmonic sparkle
            
            print(f"    ✓ {ch_id}: Harmonic series enhancement (2nd, 4th, 8th)")
            harmonics_enhanced += 1
    
    # BUS-LEVEL MIDRANGE & HIGH-FREQ PROCESSING
    print("\n🎛️ Bus-Level Processing")
    print("-" * 25)
    
    # Enhanced vocal bus processing
    if 'vocal_bus' in session.buses:
        vocal_bus = session.buses['vocal_bus']
        
        # Vocal bus presence and air
        vocal_bus.comp_enabled = True
        vocal_bus.comp_threshold = -12.0
        vocal_bus.comp_ratio = 2.5
        vocal_bus.glue_amount = 0.3  # Light glue for natural vocals
        
        print(f"    ✓ Vocal bus: Light glue compression + presence enhancement")
    
    # Enhanced instrument bus processing
    if 'instrument_bus' in session.buses:
        instrument_bus = session.buses['instrument_bus']
        
        # Instrument bus width and air
        instrument_bus.width = 1.3  # Wider for spaciousness
        instrument_bus.saturation = 0.05  # Very light harmonic enhancement
        
        print(f"    ✓ Instrument bus: Enhanced width (130%) + harmonic warmth")
    
    # FINAL SUMMARY
    print(f"\n" + "=" * 55)
    print(f"✅ MIDRANGE & HIGH-FREQUENCY MASTERY COMPLETE!")
    print(f"=" * 55)
    print(f"🧹 Mud removal: {mud_removed} channels cleaned")
    print(f"🎤 Vocal processing: {vocals_processed} channels enhanced")
    print(f"🎸 Instruments: {instruments_processed} channels optimized") 
    print(f"🤫 Harshness control: {harshness_controlled} channels tamed")
    print(f"✨ Air enhancement: {air_enhanced} channels sparkled")
    print(f"🌐 Stereo imaging: {spatial_enhanced} channels positioned")
    print(f"🎵 Harmonic enhancement: {harmonics_enhanced} channels enriched")
    print(f"\n🎯 Your mix now has professional clarity, presence, and air!")
    print(f"💫 Ready for final mastering with full-spectrum enhancement!")

# Apply the midrange and high-frequency processing
if 'session' in locals():
    apply_midrange_and_high_freq_processing(session)
    print("\n🚀 Full-spectrum professional processing complete!")
else:
    print("❌ No session found - please run the previous cells first.")

❌ No session found - please run the previous cells first.


In [9]:
# Initialize mixing session
if not channels or sum(len(tracks) for tracks in channels.values()) == 0:
    print("❌ Cannot proceed - no channels loaded!")
    print("📝 Please go back to Step 1 and update your channel directory path.")
else:
    # Initialize mixing session
    session = MixingSession(
        channels=channels,
        template=selected_template,
        template_params=template_customization,
        sample_rate=44100,
        bit_depth=24
    )
    
    # Configure mix settings with intelligent balance
    mix_settings = {
        "buses": {
            "drum_bus": {"channels": ["drums.*"], "compression": 0.8, "glue": 0.4},
            "bass_bus": {"channels": ["bass.*"], "compression": 0.8, "saturation": 0.2},
            "vocal_bus": {"channels": ["vocals.*", "backvocals.*"], "compression": 0.3, "presence": 0.5},
            "instrument_bus": {"channels": ["guitars.*", "keys.*", "synths.*"], "width": 0.7},
        },
        "sends": {},  # Simplified - no effects sends
        "master": {
            "eq_mode": "gentle",
            "compression": 0.2,
            "limiter": True,
            "target_lufs": -14,
        },
        "automation": {
            "vocal_rides": False,
            "drum_fills": False,
            "outro_fade": False,
        },
        "mix_balance": selected_balance  # 🎚️ Apply chosen balance profile
    }
    
    # Add channel overrides if they exist
    if 'channel_overrides' in locals() and channel_overrides:
        mix_settings["channel_overrides"] = channel_overrides
        print(f"🎚️ Channel overrides will be applied to mixing engine!")
    
    session.configure(mix_settings)
    print("✅ Mix configured with balance controls")

📁 Loading audio channels...
  ✓ Loaded: drums.tom
  ✓ Loaded: drums.hihat
  ✓ Loaded: drums.kick
  ✓ Loaded: drums.snare
  ✓ Loaded: drums.cymbal
  ✓ Loaded: bass.bass_guitar5
  ✓ Loaded: bass.bass1
  ✓ Loaded: bass.bass_guitar3
  ✓ Loaded: bass.bass_synth2
  ✓ Loaded: bass.bass_synth4
  ✓ Loaded: guitars.electric_guitar4
  ✓ Loaded: guitars.electric_guitar5
  ✓ Loaded: guitars.electric_guitar6
  ✓ Loaded: guitars.electric_guitar2
  ✓ Loaded: guitars.acoustic_guitar1
  ✓ Loaded: guitars.electric_guitar3
  ✓ Loaded: keys.bell3
  ✓ Loaded: keys.clavinet1
  ✓ Loaded: keys.piano4
  ✓ Loaded: keys.piano2
  ✓ Loaded: vocals.lead_vocal3
  ✓ Loaded: vocals.lead_vocal2
  ✓ Loaded: vocals.lead_vocal1
  ✓ Loaded: backvocals.lead_vocal3
  ✓ Loaded: backvocals.lead_vocal2
  ✓ Loaded: backvocals.backing_vocal
  ✓ Loaded: backvocals.lead_vocal1
  ✓ Loaded: backvocals.lead_vocal4
  ✓ Loaded: synths.rythmic_synth1
  ✓ Loaded: synths.pad3
  ✓ Loaded: synths.pad2
  ✓ Loaded: fx.perc6
  ✓ Loaded: fx.fx1
 

In [10]:
# 🔥 MODERN PARALLEL PROCESSING CHAINS
# Professional parallel compression and multi-layer processing for commercial punch

def apply_modern_parallel_processing(session):
    """Apply modern parallel compression chains and multi-layer processing"""
    
    print("🔥 APPLYING MODERN PARALLEL PROCESSING CHAINS")
    print("=" * 60)
    
    from dsp_premitives import (
        compressor, peaking_eq, shelf_filter, highpass_filter, lowpass_filter
    )
    from audio_utils import db_to_linear, ensure_stereo
    import numpy as np
    import copy
    
    # PARALLEL DRUM PROCESSING
    print("\n🥁 PARALLEL DRUM PROCESSING")
    print("-" * 35)
    
    drum_parallel_chains = 0
    
    for ch_id, strip in session.channel_strips.items():
        if strip.category.lower() == 'drums':
            print(f"\n  Processing {ch_id}...")
            
            # Create parallel processing chains
            original_settings = {
                'gain': strip.gain,
                'comp_threshold': getattr(strip, 'comp_threshold', -12.0),
                'comp_ratio': getattr(strip, 'comp_ratio', 4.0),
                'comp_attack': getattr(strip, 'comp_attack', 5.0),
                'comp_release': getattr(strip, 'comp_release', 100.0)
            }
            
            if 'kick' in ch_id.lower():
                # KICK: Three parallel chains
                
                # Chain 1: Natural/Clean (40% blend)
                # Keep existing settings for natural sound
                natural_blend = 0.4
                
                # Chain 2: Aggressive Compression (35% blend) 
                aggressive_threshold = -25.0
                aggressive_ratio = 8.0
                aggressive_attack = 0.5
                aggressive_release = 30.0
                aggressive_blend = 0.35
                
                # Chain 3: Sub Enhancement (25% blend)
                sub_threshold = -15.0
                sub_ratio = 6.0  
                sub_attack = 10.0
                sub_release = 200.0
                sub_blend = 0.25
                
                # Simulate parallel blend by weighted average
                blended_threshold = (natural_blend * original_settings['comp_threshold'] + 
                                   aggressive_blend * aggressive_threshold + 
                                   sub_blend * sub_threshold)
                blended_ratio = (natural_blend * original_settings['comp_ratio'] + 
                               aggressive_blend * aggressive_ratio + 
                               sub_blend * sub_ratio)
                blended_attack = (natural_blend * original_settings['comp_attack'] + 
                                aggressive_blend * aggressive_attack + 
                                sub_blend * sub_attack)
                blended_release = (natural_blend * original_settings['comp_release'] + 
                                 aggressive_blend * aggressive_release + 
                                 sub_blend * sub_release)
                
                # Apply blended settings
                strip.comp_enabled = True
                strip.comp_threshold = blended_threshold
                strip.comp_ratio = blended_ratio
                strip.comp_attack = blended_attack
                strip.comp_release = blended_release
                
                # Boost gain to compensate for parallel blend power
                strip.gain *= 1.3
                
                # EQ for parallel chain character
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['50hz'] = {'gain': 2.5, 'q': 1.0}    # Sub power
                strip.eq_bands['2500hz'] = {'gain': 1.8, 'q': 1.2}  # Attack punch
                strip.eq_bands['300hz'] = {'gain': -2.0, 'q': 1.5}  # Clean up mud
                
                print(f"    ✓ Kick parallel blend: Natural(40%) + Aggressive(35%) + Sub(25%)")
                
            elif 'snare' in ch_id.lower():
                # SNARE: Dual parallel chains
                
                # Chain 1: Natural (50% blend)
                natural_blend = 0.5
                
                # Chain 2: Crack Enhancement (50% blend)
                crack_threshold = -20.0
                crack_ratio = 6.0
                crack_attack = 1.0
                crack_release = 50.0
                crack_blend = 0.5
                
                # Blend settings
                strip.comp_enabled = True
                strip.comp_threshold = (natural_blend * original_settings['comp_threshold'] + 
                                      crack_blend * crack_threshold)
                strip.comp_ratio = (natural_blend * original_settings['comp_ratio'] + 
                                  crack_blend * crack_ratio)
                strip.comp_attack = (natural_blend * original_settings['comp_attack'] + 
                                   crack_blend * crack_attack)
                strip.comp_release = (natural_blend * original_settings['comp_release'] + 
                                    crack_blend * crack_release)
                
                # Parallel processing gain boost
                strip.gain *= 1.25
                
                # EQ for parallel character
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['200hz'] = {'gain': 1.2, 'q': 0.8}   # Body
                strip.eq_bands['4000hz'] = {'gain': 2.5, 'q': 1.0}  # Crack
                strip.eq_bands['10000hz'] = {'gain': 1.5, 'q': 0.6} # Air
                
                print(f"    ✓ Snare parallel blend: Natural(50%) + Crack Enhancement(50%)")
                
            elif 'hihat' in ch_id.lower() or 'cymbal' in ch_id.lower():
                # CYMBALS: Subtle parallel enhancement
                
                # Chain 1: Natural (70% blend)
                natural_blend = 0.7
                
                # Chain 2: Sparkle Enhancement (30% blend) 
                sparkle_threshold = -18.0
                sparkle_ratio = 3.0
                sparkle_attack = 2.0
                sparkle_release = 80.0
                sparkle_blend = 0.3
                
                # Blend settings
                strip.comp_enabled = True
                strip.comp_threshold = (natural_blend * original_settings['comp_threshold'] + 
                                      sparkle_blend * sparkle_threshold)
                strip.comp_ratio = (natural_blend * original_settings['comp_ratio'] + 
                                  sparkle_blend * sparkle_ratio)
                strip.comp_attack = (natural_blend * original_settings['comp_attack'] + 
                                   sparkle_blend * sparkle_attack)
                strip.comp_release = (natural_blend * original_settings['comp_release'] + 
                                    sparkle_blend * sparkle_release)
                
                # Moderate gain boost
                strip.gain *= 1.15
                
                # EQ for sparkle
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['12000hz'] = {'gain': 2.0, 'q': 0.7}  # Shimmer
                strip.eq_bands['6000hz'] = {'gain': -1.5, 'q': 2.0}  # Reduce harshness
                
                print(f"    ✓ Cymbal parallel blend: Natural(70%) + Sparkle(30%)")
                
            drum_parallel_chains += 1
    
    # PARALLEL VOCAL PROCESSING 
    print(f"\n🎤 PARALLEL VOCAL PROCESSING")
    print("-" * 35)
    
    vocal_parallel_chains = 0
    
    for ch_id, strip in session.channel_strips.items():
        if 'vocal' in strip.category.lower():
            print(f"\n  Processing {ch_id}...")
            
            original_settings = {
                'gain': strip.gain,
                'comp_threshold': getattr(strip, 'comp_threshold', -15.0),
                'comp_ratio': getattr(strip, 'comp_ratio', 3.0),
                'comp_attack': getattr(strip, 'comp_attack', 8.0),
                'comp_release': getattr(strip, 'comp_release', 120.0)
            }
            
            if 'lead_vocal' in ch_id.lower() or ('vocals' in strip.category.lower() and 'backing' not in ch_id.lower()):
                # LEAD VOCALS: Triple parallel chains
                
                # Chain 1: Natural (45% blend)
                natural_blend = 0.45
                
                # Chain 2: Presence/Power (35% blend)
                presence_threshold = -22.0
                presence_ratio = 5.0
                presence_attack = 3.0
                presence_release = 80.0
                presence_blend = 0.35
                
                # Chain 3: Intimacy/Warmth (20% blend)
                warmth_threshold = -12.0
                warmth_ratio = 2.5
                warmth_attack = 15.0
                warmth_release = 150.0
                warmth_blend = 0.20
                
                # Blend all three chains
                strip.comp_enabled = True
                strip.comp_threshold = (natural_blend * original_settings['comp_threshold'] + 
                                      presence_blend * presence_threshold + 
                                      warmth_blend * warmth_threshold)
                strip.comp_ratio = (natural_blend * original_settings['comp_ratio'] + 
                                  presence_blend * presence_ratio + 
                                  warmth_blend * warmth_ratio)
                strip.comp_attack = (natural_blend * original_settings['comp_attack'] + 
                                   presence_blend * presence_attack + 
                                   warmth_blend * warmth_attack)
                strip.comp_release = (natural_blend * original_settings['comp_release'] + 
                                    presence_blend * presence_release + 
                                    warmth_blend * warmth_release)
                
                # Significant gain boost for parallel power
                strip.gain *= 1.4
                
                # Multi-layer EQ for parallel character
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['100hz'] = {'gain': -2.0, 'q': 0.7}   # Clean lows
                strip.eq_bands['300hz'] = {'gain': 0.8, 'q': 0.8}    # Warmth
                strip.eq_bands['2500hz'] = {'gain': 2.2, 'q': 1.2}   # Presence 
                strip.eq_bands['3500hz'] = {'gain': 1.5, 'q': 0.9}   # Clarity
                strip.eq_bands['12000hz'] = {'gain': 1.2, 'q': 0.5}  # Air
                
                print(f"    ✓ Lead vocal parallel: Natural(45%) + Presence(35%) + Warmth(20%)")
                
            elif 'backing' in ch_id.lower() or 'backvocals' in strip.category.lower():
                # BACKING VOCALS: Dual parallel chains
                
                # Chain 1: Natural (60% blend) 
                natural_blend = 0.6
                
                # Chain 2: Blend Enhancement (40% blend)
                blend_threshold = -16.0
                blend_ratio = 4.0
                blend_attack = 6.0
                blend_release = 100.0
                blend_blend = 0.4
                
                # Blend settings
                strip.comp_enabled = True
                strip.comp_threshold = (natural_blend * original_settings['comp_threshold'] + 
                                      blend_blend * blend_threshold)
                strip.comp_ratio = (natural_blend * original_settings['comp_ratio'] + 
                                  blend_blend * blend_ratio)
                strip.comp_attack = (natural_blend * original_settings['comp_attack'] + 
                                   blend_blend * blend_attack)
                strip.comp_release = (natural_blend * original_settings['comp_release'] + 
                                    blend_blend * blend_release)
                
                # Moderate gain boost
                strip.gain *= 1.2
                
                # EQ for backing vocal blend
                strip.eq_bands = strip.eq_bands or {}
                strip.eq_bands['2200hz'] = {'gain': -0.8, 'q': 1.0}  # Make space for lead
                strip.eq_bands['4000hz'] = {'gain': 1.0, 'q': 0.8}   # Harmonics
                strip.eq_bands['8000hz'] = {'gain': 0.8, 'q': 0.6}   # Air
                
                print(f"    ✓ Backing vocal parallel: Natural(60%) + Blend Enhancement(40%)")
                
            vocal_parallel_chains += 1
    
    # PARALLEL BASS PROCESSING
    print(f"\n🎸 PARALLEL BASS PROCESSING")
    print("-" * 35)
    
    bass_parallel_chains = 0
    
    for ch_id, strip in session.channel_strips.items():
        if strip.category.lower() == 'bass':
            print(f"\n  Processing {ch_id}...")
            
            original_settings = {
                'gain': strip.gain,
                'comp_threshold': getattr(strip, 'comp_threshold', -15.0),
                'comp_ratio': getattr(strip, 'comp_ratio', 3.0),
                'comp_attack': getattr(strip, 'comp_attack', 10.0),
                'comp_release': getattr(strip, 'comp_release', 120.0)
            }
            
            # BASS: Triple frequency-split parallel chains
            
            # Chain 1: Natural (40% blend)
            natural_blend = 0.4
            
            # Chain 2: Sub Foundation (35% blend)  
            sub_threshold = -20.0
            sub_ratio = 4.0
            sub_attack = 15.0
            sub_release = 200.0
            sub_blend = 0.35
            
            # Chain 3: Attack/Punch (25% blend)
            attack_threshold = -18.0
            attack_ratio = 5.0
            attack_attack = 3.0
            attack_release = 60.0
            attack_blend = 0.25
            
            # Blend all three chains
            strip.comp_enabled = True
            strip.comp_threshold = (natural_blend * original_settings['comp_threshold'] + 
                                  sub_blend * sub_threshold + 
                                  attack_blend * attack_threshold)
            strip.comp_ratio = (natural_blend * original_settings['comp_ratio'] + 
                              sub_blend * sub_ratio + 
                              attack_blend * attack_ratio)
            strip.comp_attack = (natural_blend * original_settings['comp_attack'] + 
                               sub_blend * sub_attack + 
                               attack_blend * attack_attack)
            strip.comp_release = (natural_blend * original_settings['comp_release'] + 
                                sub_blend * sub_release + 
                                attack_blend * attack_release)
            
            # Gain boost for parallel power
            strip.gain *= 1.3
            
            # Frequency-specific EQ for parallel chains
            strip.eq_bands = strip.eq_bands or {}
            strip.eq_bands['60hz'] = {'gain': 2.0, 'q': 1.0}     # Sub foundation
            strip.eq_bands['400hz'] = {'gain': -1.5, 'q': 1.5}   # Clean mud
            strip.eq_bands['1500hz'] = {'gain': 1.8, 'q': 1.0}   # Attack/definition
            strip.eq_bands['3000hz'] = {'gain': 1.0, 'q': 0.8}   # Harmonics
            
            print(f"    ✓ Bass parallel: Natural(40%) + Sub Foundation(35%) + Attack(25%)")
            bass_parallel_chains += 1
    
    # ENHANCED BUS-LEVEL PARALLEL PROCESSING
    print(f"\n🎛️ BUS-LEVEL PARALLEL PROCESSING")
    print("-" * 40)
    
    # Drum bus parallel processing
    if 'drum_bus' in session.buses:
        drum_bus = session.buses['drum_bus']
        
        # Multi-stage compression simulation
        drum_bus.comp_enabled = True
        drum_bus.comp_threshold = -8.0    # First stage: gentle
        drum_bus.comp_ratio = 2.5
        drum_bus.glue_amount = 0.8        # Strong glue for parallel effect
        drum_bus.saturation = 0.25        # Parallel saturation blend
        
        print(f"    ✓ Drum bus: Multi-stage parallel compression + saturation")
    
    # Vocal bus parallel processing  
    if 'vocal_bus' in session.buses:
        vocal_bus = session.buses['vocal_bus']
        
        # Vocal bus parallel blend
        vocal_bus.comp_enabled = True
        vocal_bus.comp_threshold = -10.0  # Gentle bus compression
        vocal_bus.comp_ratio = 2.0
        vocal_bus.glue_amount = 0.4       # Light glue for natural vocals
        vocal_bus.saturation = 0.1        # Subtle harmonic enhancement
        
        print(f"    ✓ Vocal bus: Gentle parallel compression + harmonic warmth")
    
    # Bass bus parallel processing
    if 'bass_bus' in session.buses:
        bass_bus = session.buses['bass_bus']
        
        # Bass bus foundation
        bass_bus.comp_enabled = True
        bass_bus.comp_threshold = -12.0
        bass_bus.comp_ratio = 2.5
        bass_bus.glue_amount = 0.6        # Solid foundation glue
        bass_bus.saturation = 0.15        # Harmonic richness
        
        print(f"    ✓ Bass bus: Foundation parallel compression + harmonic saturation")
    
    # FINAL SUMMARY
    print(f"\n" + "=" * 60)
    print(f"🔥 MODERN PARALLEL PROCESSING COMPLETE!")
    print(f"=" * 60)
    print(f"🥁 Drum parallel chains: {drum_parallel_chains} channels enhanced")
    print(f"🎤 Vocal parallel chains: {vocal_parallel_chains} channels enhanced") 
    print(f"🎸 Bass parallel chains: {bass_parallel_chains} channels enhanced")
    print(f"\n💪 Parallel Processing Benefits Applied:")
    print(f"   • Natural dynamics + aggressive punch simultaneously")
    print(f"   • Multiple compression characters blended per source")
    print(f"   • Frequency-specific parallel enhancement")
    print(f"   • Professional bus-level parallel processing")
    print(f"   • Commercial-level power and punch")
    print(f"\n🚀 Your mix now has modern commercial-grade parallel processing!")
    print(f"🎯 Ready for professional-quality results!")

# Apply the modern parallel processing
if 'session' in locals():
    apply_modern_parallel_processing(session)
    print("\n✅ Parallel processing chains implemented successfully!")
else:
    print("❌ No session found - please run the previous cells first.")

🔥 APPLYING MODERN PARALLEL PROCESSING CHAINS

🥁 PARALLEL DRUM PROCESSING
-----------------------------------

  Processing drums.tom...

  Processing drums.hihat...
    ✓ Cymbal parallel blend: Natural(70%) + Sparkle(30%)

  Processing drums.kick...
    ✓ Kick parallel blend: Natural(40%) + Aggressive(35%) + Sub(25%)

  Processing drums.snare...
    ✓ Snare parallel blend: Natural(50%) + Crack Enhancement(50%)

  Processing drums.cymbal...
    ✓ Cymbal parallel blend: Natural(70%) + Sparkle(30%)

🎤 PARALLEL VOCAL PROCESSING
-----------------------------------

  Processing vocals.lead_vocal3...
    ✓ Lead vocal parallel: Natural(45%) + Presence(35%) + Warmth(20%)

  Processing vocals.lead_vocal2...
    ✓ Lead vocal parallel: Natural(45%) + Presence(35%) + Warmth(20%)

  Processing vocals.lead_vocal1...
    ✓ Lead vocal parallel: Natural(45%) + Presence(35%) + Warmth(20%)

  Processing backvocals.lead_vocal3...
    ✓ Lead vocal parallel: Natural(45%) + Presence(35%) + Warmth(20%)

  Pro

## 🔥 Step 3.8: Modern Parallel Processing Chains

Professional parallel compression and multi-layer processing for commercial-quality punch and dynamics.

## 🎛️ Step 4: Process Mix

Initialize mixing session and create the final mix.

In [ ]:
# Check if session was created successfully
if 'session' not in locals():
    print("❌ Cannot process mix - no session created!")
    print("📝 Please ensure channels are loaded in Step 1 and session is configured in Step 3.")
else:
    # Create output directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f"/Users/itay/Documents/post_mix_data/mixing_sessions/session_{timestamp}"
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"📁 Output directory: {output_dir}")
    print("\n🎛️ Processing mix...\n")
    
    # Process the mix - ONLY export full mix
    try:
        mix_results = session.process_mix(
            output_dir=output_dir,
            export_individual_channels=False,
            export_buses=False,
            export_stems=False,
            export_full_mix=True,  # Only export the final mix
            progress_callback=lambda msg: print(f"  {msg}")
        )
        
        print("\n✅ Mix processing complete!")
        print("\n📊 Mix Results:")
        print(f"  • Peak Level: {mix_results['peak_db']:.1f} dBFS")
        print(f"  • RMS Level: {mix_results['rms_db']:.1f} dBFS")
        print(f"  • LUFS: {mix_results['lufs']:.1f}")
        print(f"  • Dynamic Range: {mix_results['dynamic_range']:.1f} dB")
        print(f"  • Processing Time: {mix_results['time']:.1f} seconds")
        
        # Find and display the exported mix file
        mix_file = os.path.join(output_dir, "full_mix.wav")
        if os.path.exists(mix_file):
            size_mb = os.path.getsize(mix_file) / (1024 * 1024)
            if size_mb > 0.01:  # At least 10KB
                print(f"\n📁 Final Mix: {mix_file} ({size_mb:.1f} MB)")
                print("✅ Your mix is ready!")
            else:
                print(f"\n⚠️ Mix file created but is very small ({size_mb:.3f} MB)")
                print("   This usually means no input channels were processed.")
        else:
            print("\n⚠️ Mix file not found")
            
    except Exception as e:
        print(f"\n❌ Error during mixing: {e}")
        import traceback
        traceback.print_exc()

📁 Output directory: /Users/itay/Documents/post_mix_data/mixing_sessions/session_20250830_144035

🎛️ Processing mix...

  Processing individual channels...
  Processing drum_bus...
⚠️ Applied clipping protection to tom (peak was 3.8dBFS, reduced to -3.7dBFS)
⚠️ Applied clipping protection to hihat (peak was 7.5dBFS, reduced to -3.7dBFS)
⚠️ Applied clipping protection to kick (peak was 3.1dBFS, reduced to -3.7dBFS)
⚠️ Applied clipping protection to snare (peak was 8.6dBFS, reduced to -3.7dBFS)
⚠️ Applied clipping protection to cymbal (peak was 4.9dBFS, reduced to -3.7dBFS)
  Processing bass_bus...
⚠️ Applied clipping protection to bass_synth2 (peak was 6.6dBFS, reduced to -3.7dBFS)
  Processing vocal_bus...
⚠️ Applied clipping protection to lead_vocal4 (peak was 3.0dBFS, reduced to -3.7dBFS)
  Processing instrument_bus...


## 💾 Step 5: Save Session Data

Save session metadata and results for reference.

In [ ]:
# Save session information for reference  
if 'session' in locals() and 'mix_results' in locals():
    try:
        # Convert numpy values to Python types for JSON serialization
        def convert_for_json(obj):
            if isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            return obj
        
        # Clean the results for JSON serialization
        clean_results = {}
        for key, value in mix_results.items():
            clean_results[key] = convert_for_json(value)
        
        session_data = {
            "timestamp": timestamp,
            "template": selected_template,
            "template_params": template_customization,
            "mix_settings": mix_settings,
            "results": clean_results,
            "output_file": os.path.join(output_dir, "full_mix.wav"),
            "channels_processed": len([c for cat in channels.values() for c in cat.keys()])
        }
        
        session_file = os.path.join(output_dir, "mix_session.json")
        with open(session_file, 'w') as f:
            json.dump(session_data, f, indent=2)
        
        print(f"💾 Session saved: {session_file}")
        print("\n✅ Mix complete! Your final mix is ready.")
        
    except Exception as e:
        print(f"⚠️ Could not save session data: {e}")
else:
    print("⚠️ No session or results to save - please run the mixing process first.")